---
title: "Project Working File"
execute:
  kernel: ad688-venv
bibliography: references.bib
csl: csl/econometrica.csl
format: 
  html:
    toc: true
    number-sections: true
    df-print: paged
---

## This file contains the code blocks for data cleaning, EDAs, skill gap analysis, NLP processing, etc. as a working file
## Each file that contributes to the overall quarto website will contain the text and visual outputs only. 


### Code block for data_cleaning.qmd: 

In [ ]:
import pandas as pd

# Load lightcast_job_postings.csv 
df = pd.read_csv("data/lightcast_job_postings.csv")
df.head()
df.columns.tolist()

# Drop columns
columns_to_drop = [
  # tracking & other metadata
    "ID", "LAST_UPDATED_DATE", "LAST_UPDATED_TIMESTAMP", "DUPLICATES",
    "SOURCE_TYPES", "SOURCES", "URL", "ACTIVE_URLS", "ACTIVE_SOURCES_INFO", "MODELED_EXPIRED", "MODELED_DURATION", "TITLE_RAW",
  # outdated NAICS and SOC codes
    "NAICS2", "NAICS2_NAME", "NAICS3", "NAICS3_NAME",
    "NAICS4", "NAICS4_NAME", "NAICS5", "NAICS5_NAME",
    "NAICS6", "NAICS6_NAME", 
    "SOC_2", "SOC_2_NAME", "SOC_3", "SOC_3_NAME",
    "SOC_4", "SOC_4_NAME", "SOC_5", "SOC_5_NAME",
    "SOC_2021_2", "SOC_2021_2_NAME", "SOC_2021_3", "SOC_2021_3_NAME",
    "SOC_2021_5", "SOC_2021_5_NAME",
    "NAICS_2022_2", "NAICS_2022_2_NAME", "NAICS_2022_3", "NAICS_2022_3_NAME",
    "NAICS_2022_4", "NAICS_2022_4_NAME", "NAICS_2022_5", "NAICS_2022_5_NAME"
]
df.drop(columns=columns_to_drop, inplace=True)
df.info()

In [ ]:
import missingno as msno
import matplotlib.pyplot as plt

# Visualize missing data using missingno heatmap
plt.figure(figsize=(12, 6))
msno.bar(df)
plt.title("Missing Data Bar Chart")
plt.tight_layout()
plt.show()

# Identify columns that have a significant amount of missing values and sort by the percentage of missing values
missing_values_pct = (df.isna().mean() * 100).sort_values(ascending=False).reset_index()
missing_values_pct.columns = ["Column", "Missing %"]
print(missing_values_pct.to_string(index=False))

In [ ]:
# Fill in missing values for SALARY, INDUSTRY, and other relevant columns
# Fill categorical columns with "Unknown"
fill_col_unk = [
    "EXPIRED", "MSA_INCOMING", "MSA_NAME_INCOMING", "MSA", "MSA_OUTGOING", "MSA_NAME", "COMPANY_RAW", "TITLE_CLEAN", "TITLE", "TITLE_NAME", "COMPANY_NAME", "COMPANY_IS_STAFFING", "EMPLOYMENT_TYPE_NAME", "REMOTE_TYPE_NAME", "EDUCATION_LEVELS_NAME", "MIN_EDULEVELS_NAME", "SKILLS_NAME", "SPECIALIZED_SKILLS_NAME", "CERTIFICATIONS_NAME", "STATE_NAME", "CITY_NAME", "COUNTY_NAME"
]
# Loop through and fill missing values
for col in fill_col_unk:
    df[col] = df[col].fillna("Unknown")

# Do the same for relevant numerical columns, but fill with median
fill_col_median = [
    "SALARY", "SALARY_FROM", "SALARY_TO", "DURATION", "MIN_YEARS_EXPERIENCE", "MAX_YEARS_EXPERIENCE"
]
for col in fill_col_median:
    df[col] = df[col].fillna(df[col].median())

# Drop columns with >50% missing values
df.dropna(thresh=len(df) * 0.5, axis=1, inplace=True)
df.info()

In [ ]:
# Remove duplicate
df=df.drop_duplicates(subset=["TITLE_CLEAN", "COMPANY_NAME", "LOCATION", "POSTED", "EMPLOYMENT_TYPE_NAME", "REMOTE_TYPE_NAME", "SKILLS_NAME"], keep="first")

# Preview new df
df.shape

### Code block for eda.qmd: 
#### 5.1.1 Salary by Remote Work Type

In [ ]:
# 5.1.1 Visual - Compensation
import plotly.express as px
import pandas as pd

values_to_exclude = ['Unknown', '[None]']
df_filtered = df[~df['REMOTE_TYPE_NAME'].isin(values_to_exclude)]

fig1 = px.box(
    df_filtered,
    x="REMOTE_TYPE_NAME",
    y="SALARY",
    title="Salary Distribution by Work Arrangement",
    labels={"REMOTE_TYPE_NAME": "Work Arrangement", "SALARY": "Annual Salary ($)"}
)
fig1.show()

#### 5.1.2 Top Skills vs. Average Salary

In [ ]:
# 5.1.2 Visual - Skills vs. Salary
import plotly.express as px
import ast

# This function safely converts the string of a list into an actual list
def parse_skills(skill_list_str):
    try:
        return ast.literal_eval(skill_list_str)
    except (ValueError, SyntaxError):
        return []

# Create a new column with the cleaned lists of skills
df['SKILLS_LIST'] = df['SKILLS_NAME'].apply(parse_skills)

# Create a new DataFrame where each skill gets its own row
df_skills_exploded = df.explode('SKILLS_LIST')

# --- Now, create the chart using the cleaned data ---
top_10_skills_by_count = df_skills_exploded['SKILLS_LIST'].value_counts().nlargest(10).index
df_top_skills = df_skills_exploded[df_skills_exploded['SKILLS_LIST'].isin(top_10_skills_by_count)]

avg_salary_for_top_skills = df_top_skills.groupby('SKILLS_LIST')['SALARY'].mean().reset_index()

fig2 = px.bar(
    avg_salary_for_top_skills,
    x='SKILLS_LIST',
    y="SALARY",
    title="Average Salary for Top 10 Skills",
    labels={'SKILLS_LIST': "Skill", "SALARY": "Average Annual Salary ($)"}
)
fig2.show()

#### 5.1.3 Salary Distribution by Top Industries

In [ ]:
## Query Setup
# Convert the POSTED date from string to date format
df["POSTED"] = pd.to_datetime(df["POSTED"], errors="coerce")

# Create a variable for the imputed median salary
median_salary = df["SALARY"].median()

# Filter for job postings from 2024, specifically looking at Salary and Industry. Exclude unknowns, nulls, and zeros. Exclude imputed median salary. Exclude 'Unclassified Industry' 
df_jp_2024 = df[
  (df["POSTED"].dt.year==2024) & 
  (df["SALARY"] > 0) & 
  (df["SALARY"] != median_salary) &
  (df["NAICS_2022_6_NAME"]!= "Unknown") &
  (df["NAICS_2022_6_NAME"]!= "Unclassified Industry")
]

## Further filter to exclude industries that have an insignificant number of job postings
# count the number of rows per industry  
industry_jp_count = df_jp_2024["NAICS_2022_6_NAME"].value_counts()

# summarize the distribution of job counts per industry
industry_jp_count.describe()

# Set minimum threshold at 100 job postings to ensure statistical significance
top_jp_industries = industry_jp_count[industry_jp_count > 100].index

# Update df to only show top job posting industries
df_jp_2024 = df_jp_2024[df_jp_2024["NAICS_2022_6_NAME"].isin(top_jp_industries)]


## Plot: Analyze Median Salary by Industry (Seaborn)
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# group by industry name and calculate median salary, sort by descending order
top_industry_salary_order = df_jp_2024.groupby("NAICS_2022_6_NAME")["SALARY"].median().sort_values(ascending=False).head(12).index

# create bar chart
plt.figure(figsize = (14,8))
sns.barplot(
  orient='h',
  data=df_jp_2024,
  x="SALARY",
  y="NAICS_2022_6_NAME",
  order=top_industry_salary_order,
  palette="Set2",
  width=0.6,
  estimator=np.median,
  errorbar=None
)
plt.title("Median Salary by Industry in 2024", fontsize=14, weight="bold")
plt.xlabel("Median Salary ($)", fontsize=12)
plt.ylabel("Industry", fontsize=12)
plt.yticks(ha="right", fontsize=9)
plt.tight_layout()
plt.show

## Plot: Analyze Salary Distribution by Industry (Seaborn)
# determine IQRs by industry:
q25 = df_jp_2024.groupby("NAICS_2022_6_NAME")["SALARY"].quantile(0.25)
q75 = df_jp_2024.groupby("NAICS_2022_6_NAME")["SALARY"].quantile(0.75)
# sort by the middle 50% (Q3 - Q1) and name that as the new sorting order
iqr = (q75 - q25).sort_values(ascending=False).head(12)
iqr_order = iqr.index  

# Create box plot
plt.figure(figsize=(14, 12))
sns.boxplot(
  data=df_jp_2024,
  y="NAICS_2022_6_NAME",
  x="SALARY",
  order=iqr_order,
  palette="Set3",
  width=0.6
)
plt.title("Salary Distribution by Industry in 2024", fontsize=14, weight="bold")
plt.ylabel("Industry", fontsize=12)
plt.xlabel("Salary ($)", fontsize=12)
plt.yticks(ha="right", fontsize=9)
plt.grid(axis="x", linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show

#### 5.1.4 Salary Distribution by Top Industries

In [ ]:
from pyspark.sql.functions import col, lower, when

#
ai_keywords = [
    "machine learning", "data science", "artificial intelligence",
    "deep learning", "ml engineer", "ai engineer", "data engineer",
    "computer vision", "natural language processing", "nlp", "big data"
]

#
df = df.withColumn(
    "is_ai_job",
    when(
        lower(col("SOC_5_NAME")).rlike("|".join(ai_keywords)) |
        lower(col("LOT_SPECIALIZED_OCCUPATION_NAME")).rlike("|".join(ai_keywords)) |
        lower(col("NAICS_2022_6_NAME")).rlike("|".join(ai_keywords)),
        "AI"
    ).otherwise("Non-AI")
)

#
df_filtered = df.filter(
    (col("SALARY").isNotNull()) &
    (col("SALARY") > 0)
)

In [ ]:
# ----- Export All Charts -----
import plotly.express as px
import ast

fig1.write_image("chart1_salary_by_work_type.png")
fig2.write_image("chart2_skills_vs_salary.png")

print("fig1 and fig2 have been saved as PNG files.")

### code block for skill_gap_analysis.qmd: 
#### Team-based Skill Dataframe

In [ ]:
# Create list of relevant analytics skills and rate each member from 1-5
import pandas as pd

skills_data = {
    "Name": ["Angelina", "Devin", "Leo"],
    "Python": [3, 1, 3],
    "SQL": [3, 3, 3],
    "Power BI": [5, 4, 4],
    "Tableau": [4, 3, 2],
    "Excel": [5, 5, 4],
    "Machine Learning": [2, 1, 1],
    "NLP": [2, 1, 1],
    "Cloud Computing": [1, 2, 1],
    "AWS": [1, 1, 1]
}

# Convert to dataframe 
df_skills = pd.DataFrame(skills_data)
df_skills.set_index("Name", inplace=True)
df_skills

# Plot df as a heatmap to visualize skill distribution
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
sns.heatmap(df_skills, annot=True, cmap="coolwarm", linewidths=0.5)
plt.title("Team Skill Levels Heatmap")
plt.show()

#### Compare team skills to industry requirements
#### NLP Processing Code Block

In [ ]:
## Extract most in-demand skills from JD 

import pandas as pd
import re
from collections import Counter
from nltk.corpus import stopwords
from pathlib import Path
import nltk

nltk.data.path.append(str(Path.home() / "nltk_data"))

stop_words = stopwords.words("english")

# Pull description from job postings and convert into strings
job_descriptions = df["BODY"].dropna().astype(str).tolist()

## NLP processing
# Combine all JD strings into one string and convert all to lowercase 
print("Combining job descriptions...")
all_text = " ".join(job_descriptions).lower()

# Extract only alphabetical and excludes punctuation, numeric values, symbols (Tokenizing)
print("Running regex to extract words...")
words = re.findall(r'\b[a-zA]+\b', all_text)

# Filter to remove common stopwords 
print("Filtering out stopwords...")
words_filtered = [word for word in words if word not in stopwords.words("english")]

# Count the frequency of each word
print("Counting word frequencies...")
words_count = Counter(words_filtered)

# Define a list of skills: 
skills_list = {"python", "sql", "aws", "docker", "tableau", "excel", "pandas", "numpy", "power", "spark", "machine", "learning", "nlp", "cloud", "computing"}

# Extract the predefined skills that actually appear in the job postings text blob; 
skills_filtered = {
  skill: words_count[skill]
  for skill in skills_list
  if skill in words_count
}

print("Top data analytics skills from job description")
for skill, count in skills_filtered.items():
  print(f"{skill}:{count}")